# How to use ccall properly with pointers in C structs

This notebook demonstrate how to construct objects that are passed into a C library.  It is a little tricky when a field in the object is stored as a pointer to some julia allocated memory.  The question is how the memory can be preserved from being garbage collected.

See discussions at  https://discourse.julialang.org/t/how-to-keep-a-reference-for-c-structure-to-avoid-gc/9310

## C program sample

In [1]:
const output = "/tmp/test.dylib"

"/tmp/test.dylib"

In [2]:
cprog = """
#include <stdlib.h>
#include <stdio.h>

struct Bar {
 union {
   double val;
   struct Bar *ptr;
 } val;
 unsigned char ty;
};

typedef struct Bar *Barp;

Barp test1(Barp v) {
 if (v->ty == 0x00) {
   return v;
} else {
    Barp w1 = v->val.ptr;
    Barp w2 = w1 + 1;
    Barp b = (Barp) malloc(sizeof(struct Bar));
    b->ty = 0x00;
    b->val.val = w1->val.val + w2->val.val;
    /*printf(\"%f + %f = %f\\n\", w1->val.val, w2->val.val, b->val.val);*/
    return b;
 }
}
""";

In [3]:
open(`gcc -fPIC -O3 -xc -shared -o $output -`, "w") do f
     print(f, cprog)
end

## Wrong implementation (for illustrative purpose only)

In [4]:
struct Bar
    val::UInt64
    ty::UInt8
    Bar(v::Float64) = new(reinterpret(UInt64, v), 0x00)
    Bar(v::Vector{Bar}) = new(reinterpret(UInt64, pointer(v)), 0x01)
end

In [5]:
b = Bar(1.0)

Bar(0x3ff0000000000000, 0x00)

In [6]:
x = ccall((:test1, output), Ptr{Bar}, (Ptr{Bar},), Ref(b))

Ptr{Bar} @0x000000011fe0fd90

In [7]:
y = unsafe_load(x)

Bar(0x3ff0000000000000, 0x00)

In [8]:
reinterpret(Float64, y.val)

1.0

In [9]:
b2v = Bar.([2.0,3.0])

2-element Array{Bar,1}:
 Bar(0x4000000000000000, 0x00)
 Bar(0x4008000000000000, 0x00)

In [10]:
b2 = Bar(b2v)

Bar(0x000000011629cf30, 0x01)

In [11]:
x2 = ccall((:test1, output), Ptr{Bar}, (Ptr{Bar},), Ref(b2))

Ptr{Bar} @0x00007fed7b748d50

In [12]:
y2 = unsafe_load(x2)

Bar(0x4014000000000000, 0x00)

In [13]:
y2.ty

0x00

In [14]:
reinterpret(Float64, y2.val)

5.0

In [15]:
function foo(args...)
    ar = Bar.([x for x in args])
    b = Bar(ar)
    x = ccall((:test1, output), Ptr{Bar}, (Ptr{Bar},), Ref(b))
    y = unsafe_load(x)
    reinterpret(Float64, y.val)
end

foo (generic function with 1 method)

In [16]:
foo(12.0, 2.0)

14.0

In [17]:
function foo2(args...)
    ar = Bar.([x for x in args])
    b = Bar(ar)
    [rand(1000000) for _ in 1000] # try to cause gc and see if it crashes it
    x = ccall((:test1, output), Ptr{Bar}, (Ptr{Bar},), Ref(b))
    y = unsafe_load(x)
    reinterpret(Float64, y.val)
end

foo2 (generic function with 1 method)

In [18]:
# not able to crash it yet
for i in 1:1000
    foo2(1.0, 2.0)
end

## Correct implementation

The idea is to let `ccall` do the conversion.  According to the language documentation, `ccall` will first convert its arguments using [`Base.cconvert`](https://docs.julialang.org/en/stable/stdlib/c/#Base.cconvert) and [`Base.unsafe_convert`](https://docs.julialang.org/en/stable/stdlib/c/#Base.unsafe_convert).  See https://docs.julialang.org/en/stable/manual/calling-c-and-fortran-code/#Auto-conversion:-1

So here, we will pass native julia array as an argument to `ccall` and let the internal magic construct the type that the C library requires.  Using dynamic dispatch, we will define `Base.cconvert` and `Base.unsafe_convert` functions for our only pointers.

Note that everything in the return value of `cconvert` are immune from GC'ed.  In the example below, `ar` is protected during the `ccall`.

In [19]:
struct Bar2
    val::UInt64
    ty::UInt8
    Bar2(v::Float64) = new(reinterpret(UInt64, v), 0x00)
    Bar2(a::UInt64, b::UInt8) = new(a, b)
end

In [20]:
# Convert a Float64 value to a Ref{Bar2}. 
# This use the `::Float64` constructor
Base.cconvert(::Type{Ptr{Bar2}}, v::Float64) = Ref(Bar2(v)) 

# Convert a Vector{Float64} to a Ref{bar2}.
# It creates a Vector{Bar2} that holds all Bar2-wrapped Float64 values.
# Note that everything in the returned value are safe from being GC'ed,
# so in this case `ar` is immune.
function Base.cconvert(::Type{Ptr{Bar2}}, vv::Vector{Float64}) 
    ar = Bar2.(vv)
    b2 = Bar2(reinterpret(UInt64, pointer(ar)), 0x01)
    return (Ref(b2), ar)
end

# Convert the reference to a C pointer.
function Base.unsafe_convert(::Type{Ptr{Bar2}}, t::Tuple{Ref{Bar2},Vector{Bar2}})
    return Base.unsafe_convert(Ptr{Bar2}, t[1])
end

In [21]:
y = ccall((:test1, output), Ptr{Bar2}, (Ptr{Bar2},), [1.0, 2.0])
reinterpret(Float64, unsafe_load(y).val)

3.0

In [22]:
function foo3(args...)
    ccall((:test1, output), Ptr{Bar2}, (Ptr{Bar2},), [x for x in args])
end

function unpack(x)
    reinterpret(Float64, unsafe_load(x).val)
end

unpack (generic function with 1 method)

In [23]:
unpack(foo3(2.0, 4.0)) 

6.0

In [24]:
for i in 1:100000
    unpack(foo3(2.0 + i, 4.0))
end